In [1]:
import numpy as np 
import pandas as pd 
import os


In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [3]:

import torch



In [4]:
ls detect4/train/weights/best.pt

detect4/train/weights/best.pt


In [5]:
from ultralytics import YOLO

model = YOLO("detect4/train/weights/best.pt")  # You can choose a different variant like 'yolov8n.pt' or 'yolov8m.pt'



In [6]:
from datetime import timedelta
import time



def test(model_test , batch = 8,it=20):
    test_val = torch.rand(batch,3,640,640)
    torch.cpu.synchronize()
    #warm up 
    for _ in range(5):
        model_test.predict(test_val,verbose=False)
        torch.cpu.synchronize()

        
    starttime = time.perf_counter()
    for _ in range(it):
        model_test.predict(test_val,verbose=False)
        torch.cpu.synchronize()

    return timedelta(seconds=time.perf_counter()-starttime)/it


    

In [7]:
t = test(model,it=20)
t

datetime.timedelta(seconds=5, microseconds=680967)

In [8]:
from torch.quantization import quantize_dynamic
import torch

model = YOLO("detect4/train/weights/best.pt")

model_quantized = quantize_dynamic(model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)

# torch.save(model_quantized.state_dict(), 'yolov8_quantized_model.pth')

In [9]:
# model_quantized

In [10]:
t_q = test(model_quantized,it=20)
t_q

datetime.timedelta(seconds=5, microseconds=843715)

In [11]:
from copy import deepcopy
from itertools import islice

import torch
from torch import nn

# from torch.quantization.quantize_fx import convert_fx
# from torch.quantization.quantize_fx import prepare_fx
# from torch.utils.data import DataLoader



# def q_stat(model,data_loader,num_batches,device):
#     prepared_model = deepcopy(model)
#     prepared_model.eval()
#     example_input = next(iter(data_loader))
#     prepared_model = prepare_fx(
#         model=prepared_model,
#         qconfig_mapping=QCONFIG_MAPPING,
#         # example_inputs=(example_input, ),
#     )
    
#     device = torch.device(device)

#     prepared_model.eval()
#     prepared_model.to(device)

#     with torch.no_grad():
#         for image, _ in islice(data_loader, num_batches):
#             prepared_model(image.to(device))
#     prepared_model.cpu()
#     quantized_model = convert_fx(prepared_model)
#     return quantized_model

In [12]:
from ultralytics.data import YOLODataset


In [13]:
!ls annotations

images	labels


In [14]:
import yaml
with open('test.yaml', 'r') as file:
    data = yaml.safe_load(file)

In [15]:
ds = YOLODataset(data=data,img_path='annotations_s')

Scanning annotations_s/labels/train... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<00:00, 3853.23it/s]

WARNING ⚠️ annotations_s/images/train/fc3dc4d0-5952-11ec-bd4e-e7aff7c46c27.jpg: 2 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc3de866-5952-11ec-bd70-4390f6e66f66.jpg: 2 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc3e67b4-5952-11ec-be80-93f0791c872a.jpg: 2 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc3ed29e-5952-11ec-bf81-f7fa86e9085e.jpg: 4 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc3f8040-5952-11ec-84eb-0f979096c2f7.jpg: 4 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc3f8518-5952-11ec-84f8-4f5d75f45d45.jpg: 2 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc416e5a-5952-11ec-8935-635b25caae29.jpg: 4 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc418c0a-5952-11ec-897e-679888442f90.jpg: 1 duplicate labels removed
WARNING ⚠️ annotations_s/images/train/fc42defc-5952-11ec-8c0a-7b21be1bd45d.jpg: 2 duplicate labels removed
WARNING ⚠️ annotations_s/images/train

In [16]:
from ultralytics.data import build_dataloader
dl = build_dataloader(ds,batch=8,workers=8)

In [17]:
model = YOLO("detect4/train/weights/best.pt")

# model_stq.model.model = q_stat(model.model.model,dl,8,'cpu')

In [18]:
model.export(format = 'onnx') # exports the model in '.onnx' format

Ultralytics 8.3.39 🚀 Python-3.12.7 torch-2.4.1+cpu CPU (AMD Ryzen 7 7730U with Radeon Graphics)
Model summary (fused): 268 layers, 68,156,310 parameters, 0 gradients, 257.6 GFLOPs

PyTorch: starting from 'detect4/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (130.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.43...
ONNX: export success ✅ 3.9s, saved as 'detect4/train/weights/best.onnx' (260.2 MB)

Export complete (6.1s)
Results saved to /home/void/ml_oleg/detect4/train/weights
Predict:         yolo predict task=detect model=detect4/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=detect4/train/weights/best.onnx imgsz=640 data=test.yaml  
Visualize:       https://netron.app


'detect4/train/weights/best.onnx'

In [19]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = 'detect4/train/weights/best.onnx'
model_int8 = 'dynamic_quantized.onnx'

# Quantize 
quantize_dynamic(model_fp32, model_int8, weight_type=QuantType.QUInt8)

In [20]:
# ls annotations/images/test/

In [21]:
import numpy as np
from onnxruntime.quantization import CalibrationDataReader, quantize_static, QuantType, QuantFormat
import os 
import cv2
class ImageCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.idx = 0
        self.input_name = "images"

    def preprocess(self, frame):
        frame = cv2.imread(frame)
        X = cv2.resize(frame, (640, 640))
        image_data = np.array(X).astype(np.float32) / 255.0  
        image_data = np.transpose(image_data, (2, 0, 1))  
        image_data = np.expand_dims(image_data, axis=0)  
        return image_data

    def get_next(self):
        if self.idx >= len(self.image_paths):
            return None

        image_path = self.image_paths[self.idx]
        input_data = self.preprocess(image_path)
        self.idx += 1
        return {self.input_name: input_data}

calibration_image_paths  = os.listdir('annotations/images/test/')[:100]
calibration_image_paths =list(map(lambda x: f'annotations/images/test/{x}',calibration_image_paths))



In [22]:

calibration_data_reader = ImageCalibrationDataReader(calibration_image_paths)

In [23]:
quantize_static('detect4/train/weights/best.onnx', "static_quantized.onnx",
                weight_type=QuantType.QInt8,
                activation_type=QuantType.QUInt8,
                calibration_data_reader=calibration_data_reader,
                quant_format=QuantFormat.QDQ,
                nodes_to_exclude=['/model.22/Concat_3', '/model.22/Split', '/model.22/Sigmoid'
                                 '/model.22/dfl/Reshape', '/model.22/dfl/Transpose', '/model.22/dfl/Softmax', 
                                 '/model.22/dfl/conv/Conv', '/model.22/dfl/Reshape_1', '/model.22/Slice_1',
                                 '/model.22/Slice', '/model.22/Add_1', '/model.22/Sub', '/model.22/Div_1',
                                  '/model.22/Concat_4', '/model.22/Mul_2', '/model.22/Concat_5'],
                per_channel=False,
                reduce_range=True,)

In [24]:
!ls

alannn.ipynb			 nets
annotations			 out.json
annotations_s			 prepare-data-for-yolo-Copy1.ipynb
annotations_s.zip		 prepare-data-for-yolo-Copy2.ipynb
classic.ipynb			 prepare-data-for-yolo.ipynb
confusion_matrix_normalized.png  __pycache__
data.csv			 qa_test.ipynb
dert_kaggle.ipynb		 req.txt
detect				 static_quantized.onnx
detect2				 subsamples.ipynb
detect3				 testc.ipynb
detect4				 test.json
detr.ipynb			 test.png
detr.py				 test_qa.ipynb
donuts-found.jpg		 test.yaml
dynamic_quantized.onnx		 tranformers.py
eda.ipynb			 utils
evaluate.py			 weights
hw_02_pruning_solution.ipynb	 yolo.ipynb
img				 yolo-second-epoch.ipynb
img.zip				 yolo-train-fishai.ipynb
map_ids.json
